# Discrete State Seeker
Consider a robot with state variable $x = (x_0,x_1,...)$ where $x_t \in \mathcal{X}$ for every step $t \in \mathbb{N}$. At every step $t$, the robot can choose an action $a_t \in \mathcal{A}$ according to policy $a_t^\pi=\pi(x_t)$ that stochastically sets the robot's state to $x_{t+1}=Δ(x_t,a)$ and earns some reward $r(x_t,x_{t+1},a)$. The robot's goal is to maximize the total reward over an infinite time horizon with discount factor $\rho$: $\pi^\star = \argmax_{\pi}{\sum_{t=0}^{\infty} {\rho^t r(x_t, x_{t+1}, a_t^\pi)}}$.

We will use the off-policy TD algorithm (Q-learning) to estimate $Q(x,a)$, the expected reward remaining when taking action $a$ at state $x$; the $\pi^\star$ will then simply be to choose the action $a$ that maximizes $Q(x,a)$ at any state $x$. We will achieve this through:
1. Let $\alpha = (\alpha_0, \alpha_1, ...)$ be the learning rate with $\alpha_n \in (0,1)$, $\sum_n \alpha_n = \infty$, $\sum_n \alpha_n^2 < \infty$
2. Randomly generate an initial $Q_0$ matrix and an initial $x_0$. Choose some randomized policy $\pi$.
3. For $n=0,1,2,...$
    - Suppose $Q_n$ and $x_n$ are known. Choose action $a_n$ according to $\pi$ and $x_{n+1} = Δ(x_n,a_n)$.
    - Calculate updated values for $Q_{n+1}(x_n,a_n)$ based on the following equation:
        $$Q_{n+1}(x_n,a_n) = Q_n(x_n,a_n) + \alpha_n (r(x_n, x_{n+1}, a_n) + \rho \sup_{b \in \mathcal{A}} Q_n(x_{n+1},b) - Q_n(x_n,a_n))$$
4. Repeat the recursion until $Q_{n+1} \approx Q_n$.

In [135]:
import numpy as np
from matplotlib import pyplot as plt

In [180]:
class Robot:
    def __init__(self, state_space, action_space, discount, next_state, reward):
        self.state_space = state_space # \mathcal{X}  = list of possible states
        self.action_space = action_space # \mathcal{A} = list of possible actions at any state
        self.discount = discount # \rho = the discount factor
        self.next_state = next_state # Δ(x_n,a) = the callable that stochastically determines x_{n+1}
        self.reward = reward # r(x_n, x_{n+1}, a) = the reward for going from state x_n to x_{n+1} through action a

        # other instance variables
        self.Q = np.zeros(shape=(state_space.size, action_space.size))

    # resets the Q matrix to a randomized matrix
    # Alternatively, can provide a Q to set the matrix to
    # Or can provide a range to randomly select element values from
    def reset_Q(self, Q=None, range=[0,1]):
        if Q is not None:
            self.Q = np.random.rand(self.state_space.size, self.action_space.size) * (range[1] - range[0]) + range[0]

    # runs the Q-learning algorithm
    # we choose x_0 randomly and use a uniformly random policy for training
    def train_Q(self, learning_rate, min_iterations, tolerance, plot=False):
        x = np.random.choice(self.state_space)
        step = 0
        while step < min_iterations or np.abs(Q_change) > tolerance:
            # determine a and x_{n+1}
            a = np.random.choice(self.action_space)
            x_next = self.next_state(x, a)

            # find Q_{n+1}
            Q_change = learning_rate(step) * (
                self.reward(x, x_next, a) +
                self.discount * np.max(self.Q[x_next,]) -
                self.Q[x,a]
            )

            # set Q = Q_{n+1} and x = x_next
            self.Q[x, a] += Q_change
            x = x_next
            step += 1
        
        return self.Q, step

    # returns the best action to take at state x
    def best_action(self, x):
        return np.argmax(self.Q[x,])

In [181]:
example_states = ["1", "2"]
XX = np.arange(len(example_states))
example_actions = ["s", "l"]
AA = np.arange(len(example_actions))

example_p_matrix = np.array([ # action, x0, x1
    [[0.4,0.6],[0.2,0.8]],
    [[0.3,0.7],[0.1,0.9]]
])
example_r_matrix = np.array([ # action, x0, x1
    [[14,2],[2,3]],
    [[0,0],[2,6]]
])

example_bot = Robot(
    state_space = XX,
    action_space = AA,
    discount = 0.5,
    next_state = lambda x,a : np.random.choice(XX, p=example_p_matrix[a,x,:]),
    reward = lambda x,x_next,a : example_r_matrix[a,x,x_next],
)

example_bot.reset_Q(Q=np.ones(shape=(len(example_states), len(example_actions)))*np.mean(example_r_matrix))
example_bot.train_Q(
    learning_rate = lambda n : 1/(1+0.1*(n+1)),
    min_iterations = 10**6,
    tolerance = 10**(-2)
)

(array([[12.71872979,  5.86021998],
        [ 8.60730564, 11.33480161]]),
 1000000)